In [ ]:
"""
Using a Temporal Convolutional Network (TCN) to predict the movement of the stock price after the next 30 days i.e
whether the stock price will close higher or not after 30 days from the present date price.

The main reason to use TCN is its efficiency in capturing long-term dependencies and can process entire sequential data
parallely. 

To overcome the overfitting, which is major problem in using TCN, L2 regularization, early stopping and dropout is used in the model.

"""

In [15]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv1D, Dense, Flatten, BatchNormalization, Activation, Input, SpatialDropout1D, Add, Dropout
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

# Function to fetch stock data using yfinance
def fetch_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.reset_index(inplace=True)  # Reset index to make 'Date' a column
    stock_data['Date'] = pd.to_datetime(stock_data['Date'])  # Convert 'Date' to datetime format
    return stock_data[['Date', 'Close', 'Volume']]  # Return only the necessary columns

# Get user input for date range and stock ticker
start_date = input("Enter the start date (YYYY-MM-DD): ")
end_date = input("Enter the end date (YYYY-MM-DD): ")
ticker = input("Enter the stock ticker: ")

# Fetch data for the given ticker and date range
data = fetch_data(ticker, start_date, end_date)

# Function to calculate technical indicators and add them to the DataFrame
def add_technical_indicators(df):
    # Simple Moving Averages (SMA)
    df['SMA_20'] = df['Close'].rolling(window=20).mean()
    df['SMA_50'] = df['Close'].rolling(window=50).mean()
    
    # Relative Strength Index (RSI)
    delta = df['Close'].diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    # Moving Average Convergence Divergence (MACD)
    exp1 = df['Close'].ewm(span=12, adjust=False).mean()
    exp2 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = exp1 - exp2
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']
    
    # Bollinger Bands
    df['BB_middle'] = df['Close'].rolling(window=20).mean()
    df['BB_std'] = df['Close'].rolling(window=20).std()
    df['BB_upper'] = df['BB_middle'] + (df['BB_std'] * 2)
    df['BB_lower'] = df['BB_middle'] - (df['BB_std'] * 2)

    # Rolling mean and standard deviation
    df['Rolling_mean'] = df['Close'].rolling(window=20).mean()
    df['Rolling_std'] = df['Close'].rolling(window=20).std()
    
    # Quarter of the year
    df['Quarter'] = df['Date'].dt.quarter

    df.dropna(inplace=True)  # Drop rows with NaN values
    return df

# Function to create lag features
def add_lag_features(df, n_lags):
    for lag in range(1, n_lags + 1):
        df[f'Close_lag_{lag}'] = df['Close'].shift(lag)
        df[f'Volume_lag_{lag}'] = df['Volume'].shift(lag)
    df.dropna(inplace=True)  # Drop rows with NaN values after adding lag features
    return df

# Function to create sequences of data using sliding window approach
def create_sequences(data, seq_length):
    xs, ys_direction, ys_strength = [], [], []
    for i in range(len(data) - seq_length - 30):  # Adjust for 30-day prediction
        x = data.iloc[i:(i + seq_length), 1:].values  # Close, Volume, and indicators
        y_strength = data.iloc[i + seq_length + 30, 1]  # Closing price 30 days later
        y_direction = (y_strength > data.iloc[i + seq_length - 1, 1]).astype(int)  # Direction: 1 if increase, else 0
        xs.append(x)
        ys_direction.append(y_direction)
        ys_strength.append(y_strength)
    return np.array(xs), np.array(ys_direction), np.array(ys_strength)

# Add technical indicators and lag features to the data
data = add_technical_indicators(data)
data = add_lag_features(data, 5)  # Adding 5 lag features

# Function to define a residual block for the TCN model
def residual_block(x, filters, kernel_size, dilation_rate):
    shortcut = Conv1D(filters, kernel_size=1, padding='causal')(x)  # Shortcut connection
    x = Conv1D(filters, kernel_size, dilation_rate=dilation_rate, padding='causal', kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = SpatialDropout1D(0.2)(x)
    x = Conv1D(filters, kernel_size, dilation_rate=dilation_rate, padding='causal', kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Add()([shortcut, x])  # Add shortcut connection
    x = Activation('relu')(x)
    return x

# Function to build the TCN model
def build_tcn_model(input_shape):
    inputs = Input(shape=input_shape)
    x = residual_block(inputs, 64, 3, 1)  # First residual block
    x = residual_block(x, 64, 3, 2)  # Second residual block
    x = residual_block(x, 64, 3, 4)  # Third residual block
    x = Flatten()(x)  # Flatten the output

    direction_output = Dense(1, activation='sigmoid', name='direction_output')(x)  # Binary classification output
    strength_output = Dense(1, name='strength_output')(x)  # Regression output

    model = Model(inputs=inputs, outputs=[direction_output, strength_output])
    model.compile(optimizer=Nadam(), loss=['binary_crossentropy', 'mse'], metrics={'direction_output': 'accuracy', 'strength_output': 'mse'})
    return model

# Normalize the data
scaler = MinMaxScaler()
data.iloc[:, 1:] = scaler.fit_transform(data.iloc[:, 1:])

# Sequence length for capturing patterns
seq_length = 60
X, y_direction, y_strength = create_sequences(data, seq_length)

# Split data into training and testing sets
X_train, X_test, y_direction_train, y_direction_test, y_strength_train, y_strength_test = train_test_split(
    X, y_direction, y_strength, test_size=0.2, random_state=42)

# Build and train the TCN model
model = build_tcn_model((seq_length, X.shape[2]))
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.fit(X_train, [y_direction_train, y_strength_train], epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

# Evaluate the model
evaluation = model.evaluate(X_test, [y_direction_test, y_strength_test])
print(f'Test Loss: {evaluation[0]:.4f}')
print(f'Test Accuracy (Direction): {evaluation[1] * 100:.2f}%')
print(f'Test MSE (Strength): {evaluation[2]:.4f}')

# Predict for the next 30 days from end_date
last_sequence = np.array(data.iloc[-seq_length:, 1:])  # Last 60 days
last_sequence = last_sequence.reshape((1, seq_length, X.shape[2]))  # Reshape for TCN input
predicted_direction, predicted_strength = model.predict(last_sequence)
predicted_direction_class = (predicted_direction > 0.5).astype(int)
print(f'Predicted Direction for Next 30 Days: {"Up" if predicted_direction_class[0][0] == 1 else "Down"}')
print(f'Predicted Strength of Movement: {predicted_strength[0][0]:.4f}')


Enter the start date (YYYY-MM-DD):  2003-01-01
Enter the end date (YYYY-MM-DD):  2024-05-31
Enter the stock ticker:  ITC.NS


[*********************100%%**********************]  1 of 1 completed


C:\Users\HP\AppData\Local\Temp\ipykernel_17356\2534493523.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SMA_20'] = df['Close'].rolling(window=20).mean()
C:\Users\HP\AppData\Local\Temp\ipykernel_17356\2534493523.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SMA_50'] = df['Close'].rolling(window=50).mean()
C:\Users\HP\AppData\Local\Temp\ipykernel_17356\2534493523.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

Epoch 1/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 12s 33ms/step - direction_output_accuracy: 0.5772 - loss: 4.8165 - strength_output_mse: 0.6135 - val_direction_output_accuracy: 0.6087 - val_loss: 3.8930 - val_strength_output_mse: 0.0478
Epoch 2/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - direction_output_accuracy: 0.6068 - loss: 3.7827 - strength_output_mse: 0.0701 - val_direction_output_accuracy: 0.5990 - val_loss: 3.3423 - val_strength_output_mse: 0.0420
Epoch 3/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - direction_output_accuracy: 0.6440 - loss: 3.1647 - strength_output_mse: 0.0307 - val_direction_output_accuracy: 0.6280 - val_loss: 2.7524 - val_strength_output_mse: 0.0275
Epoch 4/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 3s 28ms/step - direction_output_accuracy: 0.6431 - loss: 2.6164 - strength_output_mse: 0.0195 - val_direction_output_accuracy: 0.6280 - val_loss: 2.2578 - val_strength_output_mse: 0.0200
Epoch 5/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - direction_output_accuracy: 0.6575 - 

In [ ]:
"""
The main idea for the below model is to predict the stock price movement after the next 30 days by leveraging correlation between two stocks.
The best suited model to implement it is by using tranformer model.

The model capture dependencies and correlations between different stocks at different time steps.
"""


In [17]:
import yfinance as yf
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Dropout, Flatten
from tensorflow.keras.models import Model

# Fetch data using yfinance for the given tickers and date range
def fetch_data(tickers, start_date, end_date):
    data = {}
    for ticker in tickers:
        stock_data = yf.download(ticker, start=start_date, end=end_date)
        stock_data.reset_index(inplace=True)  # Reset index to make 'Date' a column
        stock_data['Date'] = pd.to_datetime(stock_data['Date'])  # Ensure 'Date' is in datetime format
        data[ticker] = stock_data[['Date', 'Close', 'Volume']]  # Keep only relevant columns
    return data

# Add technical indicators to the dataframe
def add_technical_indicators(df):
    df['SMA_20'] = df['Close'].rolling(window=20).mean()  # 20-day simple moving average
    df['SMA_50'] = df['Close'].rolling(window=50).mean()  # 50-day simple moving average
    delta = df['Close'].diff(1)  # Difference between consecutive close prices
    gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()  # Average gains over 14 days
    loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()  # Average losses over 14 days
    rs = gain / loss  # Relative Strength
    df['RSI'] = 100 - (100 / (1 + rs))  # Relative Strength Index
    exp1 = df['Close'].ewm(span=12, adjust=False).mean()  # 12-day exponential moving average
    exp2 = df['Close'].ewm(span=26, adjust=False).mean()  # 26-day exponential moving average
    df['MACD'] = exp1 - exp2  # MACD
    df['MACD_signal'] = df['MACD'].ewm(span=9, adjust=False).mean()  # MACD signal line
    df['MACD_hist'] = df['MACD'] - df['MACD_signal']  # MACD histogram
    df['BB_middle'] = df['Close'].rolling(window=20).mean()  # Bollinger Band middle line
    df['BB_std'] = df['Close'].rolling(window=20).std()  # Bollinger Band standard deviation
    df['BB_upper'] = df['BB_middle'] + (df['BB_std'] * 2)  # Bollinger Band upper line
    df['BB_lower'] = df['BB_middle'] - (df['BB_std'] * 2)  # Bollinger Band lower line
    df.dropna(inplace=True)  # Drop rows with NaN values
    return df

# Add lag features to the dataframe
def add_lag_features(df, n_lags):
    for lag in range(1, n_lags + 1):
        df[f'Close_lag_{lag}'] = df['Close'].shift(lag)  # Lagged close price
        df[f'Volume_lag_{lag}'] = df['Volume'].shift(lag)  # Lagged volume
    df.dropna(inplace=True)  # Drop rows with NaN values
    return df

# Create sequences for the model to train on
def create_sequences(data, seq_length, target_ticker):
    if target_ticker not in data:
        raise ValueError(f"Target ticker {target_ticker} not found in the data.")
    
    xs, ys_direction, ys_strength = [], [], []
    for i in range(len(data[target_ticker]) - seq_length - 30):  # Adjust for 1-month prediction
        x = []
        for ticker in data:
            x.append(data[ticker].iloc[i:(i + seq_length), 1:].values)  # Close, Volume, and indicators
        x = np.concatenate(x, axis=1)  # Concatenate data from all tickers
        y_strength = data[target_ticker].iloc[i + seq_length:i + seq_length + 30, 1].mean()  # Average close price of next month
        y_direction = (y_strength > data[target_ticker].iloc[i + seq_length - 1, 1]).astype(int)  # Direction: 1 if increase, else 0
        xs.append(x)
        ys_direction.append(y_direction)
        ys_strength.append(y_strength)
    return np.array(xs), np.array(ys_direction), np.array(ys_strength)

# Get user input for date range and target stock
start_date = input("Enter the start date (YYYY-MM-DD): ")
end_date = input("Enter the end date (YYYY-MM-DD): ")
tickers = input("Enter the stock tickers (comma separated): ").split(',')
target_ticker = input("Enter the target stock ticker: ")

# Ensure the target ticker is included in the tickers list
if target_ticker not in tickers:
    tickers.append(target_ticker)

# Fetch data
data = fetch_data(tickers, start_date, end_date)

# Add technical indicators and lag features to each stock's data
for ticker in data:
    data[ticker] = add_technical_indicators(data[ticker])
    data[ticker] = add_lag_features(data[ticker], 5)  # Adding 5 lag features

# Normalize the data
scalers = {}
for ticker in data:
    scaler = MinMaxScaler()
    data[ticker].iloc[:, 1:] = scaler.fit_transform(data[ticker].iloc[:, 1:])  # Normalize data except the Date column
    scalers[ticker] = scaler

# Sequence length for capturing patterns
seq_length = 60
X, y_direction, y_strength = create_sequences(data, seq_length, target_ticker)

# Split data into training and testing sets
X_train, X_test, y_direction_train, y_direction_test, y_strength_train, y_strength_test = train_test_split(
    X, y_direction, y_strength, test_size=0.2, random_state=42)  # 80% training, 20% testing

# Build the Transformer model
def build_transformer_model(input_shape, num_heads=4, ff_dim=64, dropout_rate=0.1):
    inputs = Input(shape=input_shape)
    
    # Self-Attention block
    x = LayerNormalization(epsilon=1e-6)(inputs)
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=input_shape[-1])(x, x)  # Self-attention
    attn_output = Dropout(dropout_rate)(attn_output)
    x = attn_output + inputs  # Residual connection

    # Feed-forward block
    x = LayerNormalization(epsilon=1e-6)(x)
    x_ff = Dense(ff_dim, activation='relu')(x)  # Feed-forward layer
    x_ff = Dropout(dropout_rate)(x_ff)
    x_ff = Dense(input_shape[-1])(x_ff)  # Output layer
    x = x + x_ff  # Residual connection

    x = Flatten()(x)
    direction_output = Dense(1, activation='sigmoid', name='direction_output')(x)  # Binary direction output
    strength_output = Dense(1, name='strength_output')(x)  # Regression strength output

    model = Model(inputs=inputs, outputs=[direction_output, strength_output])
    model.compile(optimizer='nadam', loss=['binary_crossentropy', 'mse'], metrics={'direction_output': 'accuracy', 'strength_output': 'mse'})
    return model

# Define input shape and build the model
input_shape = (seq_length, X.shape[2])
transformer_model = build_transformer_model(input_shape)

# Use early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
transformer_model.fit(X_train, [y_direction_train, y_strength_train], epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

# Evaluate the model on the test set
evaluation = transformer_model.evaluate(X_test, [y_direction_test, y_strength_test])
print(f'Test Loss: {evaluation[0]:.4f}')
print(f'Test Accuracy (Direction): {evaluation[1] * 100:.2f}%')
print(f'Test MSE (Strength): {evaluation[2]:.4f}')

# Predict for the next month for the target stock
last_sequence = []
for ticker in data:
    last_sequence.append(data[ticker].iloc[-seq_length:, 1:].values)  # Get last 60 days of data
last_sequence = np.concatenate(last_sequence, axis=1)
last_sequence = last_sequence.reshape((1, seq_length, last_sequence.shape[1]))  # Reshape for Transformer input
predicted_direction, predicted_strength = transformer_model.predict(last_sequence)
predicted_direction_class = (predicted_direction > 0.5).astype(int)  # Convert to binary class
print(f'Predicted Direction for Next Month: {"Up" if predicted_direction_class[0][0] == 1 else "Down"}')
print(f'Predicted Strength of Movement: {predicted_strength[0][0]:.4f}')


Enter the start date (YYYY-MM-DD):  2023-01-01
Enter the end date (YYYY-MM-DD):  2024-05-31
Enter the stock tickers (comma separated):  CANBK.NS, PNB.NS
Enter the target stock ticker:  PNB.NS


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


C:\Users\HP\AppData\Local\Temp\ipykernel_17356\1347202586.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SMA_20'] = df['Close'].rolling(window=20).mean()  # 20-day simple moving average
C:\Users\HP\AppData\Local\Temp\ipykernel_17356\1347202586.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SMA_50'] = df['Close'].rolling(window=50).mean()  # 50-day simple moving average
C:\Users\HP\AppData\Local\Temp\ipykernel_17356\1347202586.py:28: SettingWithCopyWarning: 
A value is trying to be set on

Epoch 1/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 110ms/step - direction_output_accuracy: 0.8267 - loss: 13.8131 - strength_output_mse: 13.2097 - val_direction_output_accuracy: 0.7500 - val_loss: 1.1596 - val_strength_output_mse: 0.5514
Epoch 2/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - direction_output_accuracy: 0.8477 - loss: 0.9084 - strength_output_mse: 0.4704 - val_direction_output_accuracy: 0.8125 - val_loss: 0.8730 - val_strength_output_mse: 0.3024
Epoch 3/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - direction_output_accuracy: 0.8805 - loss: 0.7655 - strength_output_mse: 0.4144 - val_direction_output_accuracy: 0.8125 - val_loss: 0.8638 - val_strength_output_mse: 0.3793
Epoch 4/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - direction_output_accuracy: 0.8547 - loss: 0.7558 - strength_output_mse: 0.3900 - val_direction_output_accuracy: 0.8125 - val_loss: 0.5829 - val_strength_output_mse: 0.1178
Epoch 5/50
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - direction_output_accuracy: 0.8375 - loss: 0.7767 - str

In [16]:
import pandas as pd
import yfinance as yf

# Load the CSV file containing Nifty50 stock tickers
tickers_df = pd.read_csv('nifty50_tickers.csv')

# Extract the tickers from the DataFrame
tickers = tickers_df['Ticker'].tolist()

# Download stock data using yfinance
data = yf.download(tickers, start="2021-01-01", end="2024-01-01")['Adj Close']

# Calculate daily returns
returns = data.pct_change().dropna()

# Create the correlation matrix
correlation_matrix = returns.corr()

# Flatten the correlation matrix and create a DataFrame
correlation_data = correlation_matrix.stack().reset_index()
correlation_data.columns = ['Stock1', 'Stock2', 'Correlation']
correlation_data = correlation_data[correlation_data['Stock1'] != correlation_data['Stock2']]  # Remove self-correlation
correlation_data.reset_index(drop=True, inplace=True)

# Filter pairs with correlation >= 0.7
high_correlation_pairs = correlation_data[correlation_data['Correlation'] >= 0.8]

# Print the high correlation pairs
print(high_correlation_pairs)


[*********************100%%**********************]  100 of 100 completed
             Stock1         Stock2  Correlation
105   ADANIENSOL.NS        ATGL.NS     0.812693
200     ADANIENT.NS  ADANIPORTS.NS     0.862945
299   ADANIPORTS.NS    ADANIENT.NS     0.862945
694         ATGL.NS  ADANIENSOL.NS     0.812693
2149       CANBK.NS         PNB.NS     0.806034
7050         PNB.NS       CANBK.NS     0.806034
8300  TATAMOTORS.NS  TATAMTRDVR.NS     0.898641
8399  TATAMTRDVR.NS  TATAMOTORS.NS     0.898641
